<a href="https://colab.research.google.com/github/rajagopal17/Text-Analysis-with-Spacy/blob/master/Reviews_v(i).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', size=14)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import csv


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import gensim,pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
from gensim.models import Phrases
#from gensim.models.word2vec import sentences
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import corpora, models,similarities
from gensim.models import LdaModel


In [ ]:
from __future__ import unicode_literals
import spacy
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
#nlp=en_core_web_md.load()
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
import string
punctuations=string.punctuation

In [ ]:
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

In [ ]:
doc_file3=pd.read_csv('/content/drive/My Drive/Python/Reviews.csv',sep=',', skiprows=1,header=None)

doc_file3[9].head()

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: 9, dtype: object

##Create a new DataFrame with required columns
##Covert each row of the reviews column to string format to feed it to nlp.pipe 

In [ ]:
clean_data=pd.DataFrame(columns=['lemma_reviews','names'])

#Convert each row of reviews column to string and store it in a file

temp_file =doc_file3[9].apply(lambda x: str(x))


##Now apply nlp.pipe on 'temp_file' to do lemmatizing & entity recognition operations

In [ ]:
lemma_list=list([token.lemma_ for token in doc if token.is_stop==False] for doc in nlp.pipe(temp_file, batch_size=1000))
names_list=list([token.text for token in doc.ents] for doc in nlp.pipe(temp_file, batch_size=1000))


In [ ]:
#now assign the above nlp cleaned files to dataFrame columns

clean_data['lemma_reviews']=lemma_list
clean_data['names'] =names_list


## To get the list of all the elements of a particular column,use clean up files  'names_list', 'lemma_list'

In [ ]:
flat=[]
for x in names_list:
  for y in x:
    flat.append(y)
#flat

##Applying word2vec on the above reviews using Gensim

In [ ]:
#create dictionary: To create dicitonary need to create list and not string
dict=corpora.Dictionary(lemma_list)
#model_dict = 'reviews.dict'
#path = F'sample_drive/model_dict'
#dict.save(path)
#print(dict.token2id)

##Create Gensim model

In [ ]:
model=gensim.models.Word2Vec(lemma_list,size=300,sg=3,min_count=1,workers=3)


2020-06-13 17:53:16,656 : INFO : collecting all words and their counts
2020-06-13 17:53:16,657 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-13 17:53:16,713 : INFO : collected 18233 word types from a corpus of 288964 raw words and 6263 sentences
2020-06-13 17:53:16,714 : INFO : Loading a fresh vocabulary
2020-06-13 17:53:16,753 : INFO : effective_min_count=1 retains 18233 unique words (100% of original 18233, drops 0)
2020-06-13 17:53:16,754 : INFO : effective_min_count=1 leaves 288964 word corpus (100% of original 288964, drops 0)
2020-06-13 17:53:16,822 : INFO : deleting the raw counts dictionary of 18233 items
2020-06-13 17:53:16,823 : INFO : sample=0.001 downsamples 35 most-common words
2020-06-13 17:53:16,824 : INFO : downsampling leaves estimated 220572 word corpus (76.3% of prior 288964)
2020-06-13 17:53:16,878 : INFO : estimated required memory for 18233 words and 300 dimensions: 52875700 bytes
2020-06-13 17:53:16,879 : INFO : resetting laye

##Train the model

In [ ]:
model.train(lemma_list, total_examples=len(lemma_list), epochs=200)

2020-06-13 18:03:29,510 : WARNING : Effective 'alpha' higher than previous training cycles
2020-06-13 18:03:29,512 : INFO : training model with 3 workers on 18233 vocabulary and 300 features, using sg=3 hs=0 sample=0.001 negative=5 window=5
2020-06-13 18:03:30,545 : INFO : EPOCH 1 - PROGRESS: at 48.09% examples, 104999 words/s, in_qsize 5, out_qsize 0
2020-06-13 18:03:31,443 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-13 18:03:31,463 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-13 18:03:31,516 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-13 18:03:31,517 : INFO : EPOCH - 1 : training on 288964 raw words (220671 effective words) took 2.0s, 111079 effective words/s
2020-06-13 18:03:32,603 : INFO : EPOCH 2 - PROGRESS: at 52.45% examples, 105631 words/s, in_qsize 5, out_qsize 0
2020-06-13 18:03:33,405 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-13 18:03:33,471 : INFO : w

(44114005, 57792800)

##Save the model

In [ ]:
#mysavedmodel=model.save_word2vec_format

##Test the model

In [ ]:
model.wv.most_similar('pasta')